In [7]:
# Importação do Pyomo
from pyomo.environ import *

In [8]:
model = ConcreteModel()

In [9]:
# Dados do problema
n = 2 # Quantidade de clientes (Esse valor não varia)
m = 2 # Quantidade de requisitos (Essa variar também)
L = 100 # Limitação do custo de implementação da empresa
v = [random.randint(1, 10) for _ in range(n)] # Importância do cliente para a empresa
c = [random.randint(1, 10) for _ in range(m)] # Custo de cada requisito
dm = [[random.randint(0, 1) for _ in range(m)] for _ in range(m)] # Matriz de associação de requisitos
cm = [[random.randint(0, 1) for _ in range(m)] for _ in range(n)]  # Matriz de associação de clientes

In [10]:
# Variáveis de Decisão
model.x = Var(range(m), domain=Binary) # Requisito implementado
model.y = Var(range(n), domain=Binary) # Cliente a ser atendido

In [11]:
# Função Objetivo
model.obj = Objective(expr = sum(model.y[i] * v[i] for i in range(n)), sense = maximize)

In [12]:
# Restrição de Limitação de Custo
model.add_component(f'costTotalConstraint-', Constraint(expr = sum(model.x[i] * c[i] for i in range(m)) <= L))

In [13]:
# Restrição de dependência entre requisitos 
for i in range(m):
    for j in range(m):
        if dm[i][j] == 1:
            # In this case, requirement j depends on requirement i, so x_i >= x_j
            model.add_component(f'reqDependency-{i}-{j}', Constraint(expr=model.x[i] >= model.x[j]))

In [14]:
# Restrição para garantir a implementação de todos os requisitos do cliente selecionado
for cli in range(n):
    for req in range(m):
        if cm[cli][req] == 1:
            # In this case, requirement "req" is associated to client "cli", so x_req >= y_cli
            model.add_component(f'reqClientValidity-{cli}-{req}', Constraint(expr=model.x[req] >= model.y[cli]))

In [15]:
# Resolve o modelo
solver = SolverFactory('glpk')
results = solver.solve(model, timelimit = 100).write()
print(f'\n\nSelected clients')
for i in range(n):
    print(model.y[i]())

print(f'\n\nRequirements for the next release')
for i in range(m):
    print(model.x[i]())
    
print(f'Objective function: {model.obj.expr()}')

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16.0
  Upper bound: 16.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 4
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.02534031867980957
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------------